# Setup

The main focus of this notebook is to analyze and mitigate potential biases in a dataset. The notebook uses various techniques and technologies, such as [AIF360](https://aif360.mybluemix.net), [Fairlearn](https://fairlearn.org) , and [scikit-learn](https://scikit-learn.org/stable/), to preprocess and analyze the data, and to train and evaluate machine learning models. The notebook also includes visualizations and statistics to help understand the distribution and correlations of the data, and to identify any potential biases.

**PLEASE NOTE**: The notebook must be configured with a dataset and some configuration variables.  
The ***protected attributes*** must be categorical and binary (0,1), but the original column must be mantained as a continuous variable.

## Colab
In order to run this notebook in Google Colab, you have to:
1. Upload the project folder to your Google Drive
2. Mount your Google Drive in the next code cell
3. Update the `path_to_project` variable, with the path to the project folder in your Google Drive (e.g. `path_to_project = '/content/drive/MyDrive/Colab/project'`), this can be seen from the file tab on the left of the Colab interface
4. Update the pip installation command  (e.g. `!pip install -r /content/drive/MyDrive/Colab/project/requirements.txt`)

In [1]:
try:
  from google.colab import drive
  drive.mount('/content/drive')
  import sys
  path_to_project = '/content/drive/MyDrive/FairAlgorithm'  #UPDATE THIS LINE
  sys.path.append(path_to_project)
  !sudo apt install libcairo2-dev pkg-config python3-dev
  %pip install -r /content/drive/MyDrive/FairAlgorithm/source/requirements.txt  #UPDATE THIS LINE
  IN_COLAB = True
except:
  IN_COLAB = False

Mounted at /content/drive
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
python3-dev is already the newest version (3.10.6-1~22.04.1).
python3-dev set to manually installed.
The following packages were automatically installed and are no longer required:
  libbz2-dev libpkgconf3 libreadline-dev
Use 'sudo apt autoremove' to remove them.
The following additional packages will be installed:
  libblkid-dev libblkid1 libcairo-script-interpreter2 libffi-dev
  libglib2.0-dev libglib2.0-dev-bin libice-dev liblzo2-2 libmount-dev
  libmount1 libpixman-1-dev libselinux1-dev libsepol-dev libsm-dev
  libxcb-render0-dev libxcb-shm0-dev
Suggested packages:
  libcairo2-doc libgirepository1.0-dev libglib2.0-doc libgdk-pixbuf2.0-bin
  | libgdk-pixbuf2.0-dev libxml2-utils libice-doc cryptsetup-bin libsm-doc
The following packages will be REMOVED:
  pkgconf r-base-dev
The following NEW packages will be installed:
  libblkid-dev libcairo-script-interpreter2 

In [2]:
# import numpy as np
# import pandas as pd
# np.random.seed(1234)
# from rich import print
# from rich.columns import Columns
# from rich.panel import Panel
# from rich.align import Align
# from source.utils.print_util import *
# from source.utils.data_preprocessing import *
# import matplotlib.pyplot as plt
# from fairlearn.metrics import  MetricFrame, count, false_negative_rate, false_positive_rate, selection_rate, equalized_odds_difference, demographic_parity_difference
# from fairlearn.preprocessing import CorrelationRemover
# from fairlearn.adversarial import AdversarialFairnessClassifier
# from fairlearn.reductions import ExponentiatedGradient, Moment
# from fairlearn.postprocessing import ThresholdOptimizer
# from aif360.datasets import BinaryLabelDataset, StructuredDataset, StandardDataset
# from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric, Metric
# from aif360.algorithms.preprocessing import DisparateImpactRemover, Reweighing, LFR, OptimPreproc
# from aif360.algorithms.preprocessing.optim_preproc_helpers.opt_tools import OptTools
# from aif360.algorithms.inprocessing import PrejudiceRemover, AdversarialDebiasing, ExponentiatedGradientReduction
# from aif360.algorithms.postprocessing import RejectOptionClassification, CalibratedEqOddsPostprocessing, EqOddsPostprocessing
# from aif360.algorithms import Transformer
# import tensorflow.compat.v1 as tf
# tf.disable_eager_execution()
# from tqdm.notebook import tqdm
# import pickle

In [3]:
# from sklearn.model_selection import cross_validate, cross_val_score, cross_val_predict, train_test_split, StratifiedKFold
# from sklearn.metrics import classification_report, recall_score, accuracy_score, precision_score, confusion_matrix, roc_curve
# from sklearn.linear_model import LogisticRegression
# from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier, BaggingClassifier
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.decomposition import PCA
# from sklearn.preprocessing import StandardScaler, normalize

In [4]:
from source.utils.config import *
from source.utils.mitigation_utils import *

pip install 'aif360[inFairness]'


In [5]:
!pip install BlackBoxAuditing

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 22.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for BlackBoxAuditing: filename=BlackBoxAuditing-0.1.54-py2.py3-none-any.whl size=1394756 sha256=638b1525a7d3aa3cffd3f75bf2f935b8a336f5404eb9301c3dc783aa7eb94289
  Stored in directory: /root/.cache/pip/wheels/c0/4f/b1/80e1b0790df07536470758fe0a4f9ff8fa942fd9fe30bbb192
Successfully built BlackBoxAuditing


## Configure the notebook


In the next code cell, set all the variables that are used throughout the notebook.  
The variables are used to configure the notebook, and to set the paths to the data files.

Modify the variables in the next code cell to configure the notebook

- `dataset_name`: The name of the dataset file.
- `dataset_path`: The path to the dataset file.
- `target`: The target feature to predict.
- `target_variable_labels`: The labels for the target feature.
- `sensible_attribute`: The sensible attribute to use for bias mitigation.

In [ ]:
#INPUT
dataset_name = "myocardial-infarction"

if dataset_name == "myocardial-infarction":
  ignore_cols = []
  target_variable = 'LET_IS_cat'
  target_variable_labels= [1,0]
  sensible_attribute = 'SEX'

  default_mappings = {
      'label_maps': [{1: 'Complication', 0: 'No Alzheimer'}],
      'protected_attribute_maps': [{0: 'Female', 1: 'Male'}]
  }
elif dataset_name == 'alzheimer-disease':
  ignore_cols = []
  target_variable = 'Diagnosis'
  target_variable_labels= [1, 0]
  sensible_attribute = 'Gender_cat'
  #sensible_attribute = 'Ethnicity_cat'
  default_mappings = {
      'label_maps': [{1: 'Alzheimer', 0: 'No Alzheimer'}],
      'protected_attribute_maps': [{"Ethnicity_cat":{1: "Caucasian", 0: "Non-Caucasian"}}, {"Gender_cat":{1: "Male", 0: "Female"}}]
	}

dataset_path = path_to_project + '/data/preprocessed/preprocessed-{}.csv'.format(dataset_name) if IN_COLAB else 'data/preprocessed/preprocessed-{}.csv'.format(dataset_name)
df = pd.read_csv(dataset_path)
df = df.drop(columns=ignore_cols)
feature_cols = df.columns

In [ ]:
n_estimators = 30
random_seed = 1234
n_splits= 10

models = {'Logistic Regression':LogisticRegression(max_iter=500),
          'Decision Tree':DecisionTreeClassifier(max_depth=None),
          'Bagging':BaggingClassifier(DecisionTreeClassifier(max_depth=3),n_estimators=n_estimators),
          'Random Forest':RandomForestClassifier(n_estimators=n_estimators),
          'Extremely Randomized Trees':ExtraTreesClassifier(n_estimators=n_estimators),
          'Ada Boost':AdaBoostClassifier(DecisionTreeClassifier(max_depth=3),n_estimators=n_estimators)}

In [ ]:
config = {}
config['df']= df
config['target_variable'] = target_variable
config['sensible_attribute'] = sensible_attribute
config['path_to_project'] = path_to_project
config['n_splits'] = n_splits
config['models'] = models
config['n_estimators'] = n_estimators
config['random_seed'] = random_seed

In [ ]:
def unpack_config(config):
  return config['df'], config['target_variable'], config['sensible_attribute'], config['path_to_project'], config['n_splits'], config['models'], config['n_estimators'],config['random_seed']

## Load the data
Data is loaded from the file specified in the `dataset_path` variable using the `pandas` library.  
[pandas](https://pandas.pydata.org) is a fast, powerful, flexible and easy to use open source data analysis and manipulation tool,
built on top of the Python programming language.

In [ ]:
df.head(5)

,LET_IS_cat,ID,RAZRIV,K_SH_POST,AGE,ZSN_A,NITR_S,NA_R_1_n,R_AB_3_n,STENOK_AN,...,FK_STENOK,REC_IM,zab_leg_02,MP_TP_POST,lat_im,IBS_POST,nr_04,NOT_NA_2_n,R_AB_2_n,SEX
0,0,1118,0,0.0,57,0.0,0.0,0.0,0.0,6,...,2.0,0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1
1,0,705,0,0.0,44,0.0,0.0,0.0,0.0,0,...,0.0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1
2,0,1329,0,0.0,59,0.0,0.0,0.0,0.0,6,...,2.0,1,0.0,0.0,1.0,2.0,0.0,0.0,0.0,1
3,0,32,0,0.0,65,0.0,0.0,0.0,0.0,0,...,0.0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1
4,0,278,0,0.0,74,1.0,0.0,0.0,0.0,0,...,0.0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0


## Mitigation utils


In [ ]:
def train_test_splitting(df, n_splits):
  df_splitting = {}

  w = int(len(df)/n_splits)
  window = w
  start_point = 0
  for i in range(0,n_splits):
      train = {}
      test = {}
      df_train_1 = {}
      df_train_2 = {}
      df_test = df[start_point:window]
      if i != 0:
        df_train_1 = df[0: start_point]

      if i != n_splits-1:
        df_train_2 = df[window: len(df)]

      if (i != 0 and  i != n_splits-1):
        concat_df = [df_train_1, df_train_2]
        df_train = pd.concat(concat_df)
      elif i != 0:
        df_train = df_train_1
      else:
        df_train = df_train_2

      start_point= window
      window = window + w

      df_splitting[i] = {'train': df_train, 'test': df_test}
  return df_splitting

In [ ]:
#print(train_test_splitting(df, n_splits))

In [ ]:
def df_X_Y_split(df_train, df_test, target_variable):
  Y_train = df_train[target_variable]
  X_train = df_train.drop(target_variable, axis=1)
  Y_test = df_test[target_variable]
  X_test = df_test.drop(target_variable, axis=1)
  return X_train, Y_train, X_test, Y_test

In [ ]:
def df_X_Y_split_2D(df_train, df_test, target_variable, sensible_attribute):
  Y_train = df_train[target_variable].values
  S_train = df_train[sensible_attribute].values
  X_train = df_train.drop(target_variable, axis=1)
  X_train = X_train.drop(sensible_attribute, axis=1)

  Y_test = df_test[target_variable]
  X_test = df_test.drop(target_variable, axis=1)
  S_test = df_test[sensible_attribute].values

  X_train = normalize(X_train)
  X_test = normalize(X_test)

  pca = PCA(n_components=2)
  X_train2D = pca.fit(X_train).transform(X_train)
  X_test2D = pca.fit(X_test).transform(X_test)
  return X_train2D, Y_train, X_test2D, Y_test, S_train, S_test

In [ ]:
def df_X_Y_split_S(df_train, df_test, target_variable, sensible_attribute):
  Y_train = df_train[target_variable].values
  S_train = df_train[sensible_attribute].values
  X_train = df_train.drop(target_variable, axis=1)

  Y_test = df_test[target_variable]
  X_test = df_test.drop(target_variable, axis=1)
  S_test = df_test[sensible_attribute].values

  return X_train, Y_train, X_test, Y_test, S_train, S_test

In [ ]:
def compute_predictions_and_tests(df, target_variable, n_splits, models, n_estimators, random_seed):
  predicted_and_real_values = {}
  for model_name in tqdm(models):
    clf = models[model_name]
    df_splitting = train_test_splitting(df, n_splits)
    pred_and_y = {}
    for i in range(0,n_splits):
      df_split = df_splitting[i]
      df_train = df_split['train']
      df_test = df_split['test']

      X_train, Y_train, X_test, Y_test = df_X_Y_split(df_train, df_test, target_variable)
      clf.fit(X_train,Y_train)
      y_pred = clf.predict(X_test)

      S_test = X_test[sensible_attribute].values
      pred_and_y[i] = {'y_test': Y_test.to_numpy().astype(int), 'y_pred': y_pred.astype(int), 's_test':  S_test.astype(int)}

    predicted_and_real_values[model_name] = pred_and_y

  return predicted_and_real_values


In [ ]:
def fl_am_compute_predictions_and_tests(df, target_variable, sensible_attribute, n_splits, classifier):
  predicted_and_real_values = {}

  df_splitting = train_test_splitting(df, n_splits)
  pred_and_y = {}
  for i in range(0,n_splits):

    df_split = df_splitting[i]
    df_train = df_split['train']
    df_test = df_split['test']
    ##Input data should be in 2-dimensions
    X_train2D, Y_train, X_test2D, Y_test, S_train, S_test = df_X_Y_split_2D(df_train, df_test, target_variable, sensible_attribute)
    #print(len(X_train2D), len(Y_train),len(S_train))

    #classifier is given in in-processing mitigation
    predictor = classifier.fit(X_train2D, Y_train, sensitive_features=S_train)
    y_pred = predictor.predict(X_test2D)
    pred_and_y[i] = {'y_test': Y_test.to_numpy(), 'y_pred': y_pred, 's_test':  S_test }

  predicted_and_real_values = pred_and_y

  return predicted_and_real_values

In [ ]:
def fl_to_compute_predictions_and_tests(df, target_variable, sensible_attribute, n_splits, classifier):
  predicted_and_real_values = {}

  df_splitting = train_test_splitting(df, n_splits)
  pred_and_y = {}
  for i in range(0,n_splits):

    df_split = df_splitting[i]
    df_train = df_split['train']
    df_test = df_split['test']
    ##Input data should be in 2-dimensions
    X_train, Y_train, X_test, Y_test, S_train, S_test = df_X_Y_split_S(df_train, df_test, target_variable, sensible_attribute)
    #print(len(X_train2D), len(Y_train),len(S_train))

    #classifier is given in in-processing mitigation
    predictor = classifier.fit(X_train, Y_train, sensitive_features=S_train)
    y_pred = predictor.predict(X_test, sensitive_features=S_test)
    pred_and_y[i] = {'y_test': Y_test.to_numpy(), 'y_pred': y_pred, 's_test':  S_test}

  predicted_and_real_values = pred_and_y

  return predicted_and_real_values

In [ ]:
#print(compute_predictions(df, target_variable, n_splits, models, n_estimators, random_seed))

In [ ]:
def save_predictions_and_tests(predicted_and_test, dataset_name, sensible_attribute, mitigation,path_to_project):
  save_path = '/content/drive/MyDrive/Colab Notebooks/prediction/pred_test-{}-{}-{}.p'.format(dataset_name, sensible_attribute, mitigation)
  with open(save_path, 'wb') as fp:
    pickle.dump(predicted_and_test, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
def save_mitigated_dataset(mitigated_dataset, path_to_project,dataset_name, sensible_attribute, mitigation):
  mitigated_dataset.to_csv("{}/mitigation/mitigated-{}-{}-{}.csv".format('/content/drive/MyDrive/Colab Notebooks/',dataset_name, sensible_attribute, mitigation), sep=',', index=False, encoding='utf-8')

In [ ]:
def check_results(results):
  for model_name in tqdm(models):
    for i in range(0,n_splits):
      y_test = results[model_name][i]['y_test']
      y_pred = results[model_name][i]['y_pred']
      for j in range(0,67):
        if y_test[j] != y_pred[j]:
          print(y_test[j], y_pred[j])

# Fairlearn
[Fairlearn](https://fairlearn.org) is an open source toolkit that empowers developers of artificial intelligence (AI) systems to assess their system's fairness and mitigate any observed unfairness issues.

### Correlation Remover
Pre-processing: mitigated dataset changes

MITIGATED DATASET

ALPHA PARAMETER SET TO DEFAULT

NO MODEL ENFORCEMENT

In [ ]:
def fl_cr(config):
  mitigation='fl-cr'
  df, target_variable,sensible_attribute, path_to_project, n_splits, models, n_estimators, random_seed = unpack_config(config)
  X_raw = df.drop(target_variable, axis=1)
  X_raw = pd.get_dummies(X_raw)
  y = df[target_variable].values

  cr = CorrelationRemover(sensitive_feature_ids=[sensible_attribute])
  X_cr = cr.fit_transform(X_raw)

  X_cr = pd.DataFrame(
      X_cr, columns = X_raw.drop(sensible_attribute, axis=1).columns
  )

  X_cr[sensible_attribute] = X_raw[sensible_attribute]
  mit_fl_cr = X_cr.copy(deep=True)
  mit_fl_cr[target_variable] = y

  save_mitigated_dataset(mit_fl_cr,path_to_project,dataset_name, sensible_attribute, mitigation)
  predictions_and_tests = compute_predictions_and_tests(mit_fl_cr, target_variable, n_splits, models, n_estimators, random_seed)
  save_predictions_and_tests(predictions_and_tests, dataset_name, sensible_attribute, mitigation,path_to_project)
  return mit_fl_cr, predictions_and_tests

In [ ]:
print(fl_cr(config))

### Threshold Optimizer
Post-processing: change predictions, mitigated dataset is equal to the original one

SET THE MODEL TO CLASSIFIER - NO MODEL ENFORCEMENT

NO PARAMETERS

NO MITIGATED DATASET

In [ ]:
def fl_to(config):
  mitigation='fl-to'
  df, target_variable, sensible_attribute, path_to_project, n_splits, models, n_estimators, random_seed = unpack_config(config)
  predictions_and_tests= {}

  for model_name in tqdm(models):
    classifier =  models[model_name]
    TO = ThresholdOptimizer(estimator=classifier)
    pred_and_y = fl_to_compute_predictions_and_tests(df, target_variable, sensible_attribute, n_splits, TO)
    predictions_and_tests[model_name] = pred_and_y

  save_predictions_and_tests(predictions_and_tests, dataset_name, sensible_attribute, mitigation, path_to_project)
  return predictions_and_tests

In [ ]:
print(fl_to(config))

# AIF360
[AIF360](https://aif360.mybluemix.net) is a system used in order to detect and mitigate a possible bias inside a dataset.
It requires firstly the identification of the corresponding protected attribute (sex, race, age etc.)

### Configuration

In order to use [AIF360](https://aif360.mybluemix.net) we need to define a Binary Dataset and set the privileged/unprivileged groups

In [ ]:
data_orig_aif = BinaryLabelDataset(
    favorable_label = 1,
    unfavorable_label = 0,
    df = df.copy(),
    label_names = [target_variable],
    protected_attribute_names = [sensible_attribute])

privileged_groups = [{'SEX': 1}]  #todo mettere sensible_attribute
unprivileged_groups = [{'SEX': 0}]

## Pre-processing

### Reweighing

Pre-processing: dataset is equal to the original one, the predictor is the same, only use different weights

NO PARAMETER

NO MITIGATED DATASET

NO MODEL ENFORCEMENT

In [ ]:
def aif360_rw(config):
  mitigation = 'aif360-rw'
  df, target_variable,sensible_attribute, path_to_project, n_splits, models, n_estimators, random_seed = unpack_config(config)

  predictions_and_tests = {}
  for model_name in models:
    clf = models[model_name]
    RW = Reweighing(unprivileged_groups = unprivileged_groups,
                privileged_groups = privileged_groups)

    df_splitting = train_test_splitting(df, n_splits)
    pred_and_y = {}
    for i in range(0,n_splits):
      df_split = df_splitting[i]
      df_train = df_split['train']
      df_test = df_split['test']

      X_train, Y_train, X_test, Y_test = df_X_Y_split(df_train, df_test, target_variable)
      S_test = df_test[sensible_attribute].values

      data_orig_aif = BinaryLabelDataset(favorable_label = 1, unfavorable_label = 0, df = df_train.copy(), label_names = [target_variable], protected_attribute_names = [sensible_attribute])
      rw_dataset = RW.fit_transform(data_orig_aif)
      rw_df = rw_dataset.convert_to_dataframe()
      sample_weights= rw_df[1]["instance_weights"]


      clf.fit(X_train,Y_train,sample_weight=sample_weights)
      y_pred = clf.predict(X_test)
      pred_and_y[i] = {'y_test': Y_test.to_numpy().astype(int), 'y_pred': y_pred.astype(int), 's_test':  S_test.astype(int)}

      predictions_and_tests[model_name] = pred_and_y

  save_predictions_and_tests(predictions_and_tests, dataset_name, sensible_attribute, mitigation, path_to_project)

  return predictions_and_tests

In [ ]:
print(aif360_rw(config))

### Disparate Impact Remover

Pre-processing: It transforms the dataset in a new mitigated one

PARAMETER REPAIR LEVEL

MITIGATED DATASET

NO MODEL ENFORCEMENT

In [ ]:
def aif360_di(config):
  mitigation = 'aif360-di'
  df, target_variable,sensible_attribute, path_to_project, n_splits, models, n_estimators, random_seed = unpack_config(config)

  predictions_and_tests = {}

  #levels = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
  DIR = DisparateImpactRemover(repair_level=0.5)
  data_orig_aif = BinaryLabelDataset(favorable_label = 1, unfavorable_label = 0, df = df.copy(), label_names = [target_variable], protected_attribute_names = [sensible_attribute])

  train_repd = DIR.fit_transform(data_orig_aif)
  mit_aif360_di = train_repd.convert_to_dataframe()[0]

  save_mitigated_dataset(mit_aif360_di,path_to_project,dataset_name, sensible_attribute, mitigation)

  predictions_and_tests = compute_predictions_and_tests(mit_aif360_di, target_variable, n_splits, models, n_estimators, random_seed)

  save_predictions_and_tests(predictions_and_tests, dataset_name, sensible_attribute, mitigation, path_to_project)

  return predictions_and_tests

In [ ]:
print(aif360_di(config))

### Learning Fair Representation

PARAMETERS k, Ax, Ay, Az maxitem e maxfun

MITIGATED DATASET

NO MODEL ENFORCEMENT

In [ ]:
def aif360_lfr(config):
  mitigation = 'aif360-lfr'
  df, target_variable,sensible_attribute, path_to_project, n_splits, models, n_estimators, random_seed = unpack_config(config)

  predictions_and_tests = {}

  TR = LFR(unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups,
    seed= random_seed, k=10, Ax=0.01, Ay=1.0, Az=50.0,
    verbose=1)
  data_orig_aif = BinaryLabelDataset(favorable_label = 1, unfavorable_label = 0, df = df.copy(), label_names = [target_variable], protected_attribute_names = [sensible_attribute])

  TR = TR.fit(data_orig_aif, maxiter=5000, maxfun=5000)
  transf_dataset = TR.transform(data_orig_aif)
  mit_aif360_lfr = transf_dataset.convert_to_dataframe()[0]

  save_mitigated_dataset(mit_aif360_lfr,path_to_project,dataset_name, sensible_attribute, mitigation)

  predictions_and_tests = compute_predictions_and_tests(mit_aif360_lfr, target_variable, n_splits, models, n_estimators, random_seed)

  save_predictions_and_tests(predictions_and_tests, dataset_name, sensible_attribute, mitigation, path_to_project)

  return predictions_and_tests

In [ ]:
results= aif360_lfr(config)
print(results)

step: 0, loss: 700.2110816688185, L_x: 69933.81853139412,  L_y: 0.8635402357117458,  L_z: 0.0001871223833077862
step: 250, loss: 699.5627696234146, L_x: 69909.02862818337,  L_y: 0.4636651246245638,  L_z: 0.00017636433912653765
step: 500, loss: 697.701333263156, L_x: 69741.2869196062,  L_y: 0.28277974592369237,  L_z: 0.00011368642340817172
step: 750, loss: 281.09180556053457, L_x: 27579.084898549852,  L_y: 4.747055903807364,  L_z: 0.011078013424574063
step: 1000, loss: 694.943932228352, L_x: 69458.87702665053,  L_y: 0.3480112986919992,  L_z: 0.0001430132630936785
step: 1250, loss: 455.3419423042026, L_x: 45239.24329454142,  L_y: 2.5643973932856174,  L_z: 0.007702239310054329
step: 1500, loss: 88.93807228758594, L_x: 8579.712970214503,  L_y: 2.522004901862424,  L_z: 0.01237875367156965
step: 1750, loss: 290.1333912832927, L_x: 28910.186559493515,  L_y: 0.19088337936577943,  L_z: 0.016812846179835788
step: 2000, loss: 150.01045661913776, L_x: 14873.393727536233,  L_y: 0.2739200531724519, 

  0%|          | 0/6 [00:00<?, ?it/s]

ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 0.0

### Optimized Preprocessing

Pre-processing: mitigated dataset
Richiede valori binari come input quindi è stato fatto un pre-processing sui valori per essere 0,1

CREATE OptTool for each dataset

PARAMETERS distortion_fun, epsilon, clist, dlist

MITIGATED DATASET

NO MODEL ENFORCEMENT

In [ ]:
def get_distortion_diabetes(vold,vnew):
        bad_val=3
        OutNew=vnew["Outcome"]
        OutOld=vold["Outcome"]
        GluNew=vnew["Glucose"]
        GluOld=vold["Glucose"]
        InsOld=vold["Insulin"]
        InsNew=vnew["Insulin"]

        if ((OutNew>OutOld)& (InsNew<InsOld)): #| ((OutNew>OutOld) & (GluNew<GluOld)):
            return bad_val
        else:
            return 0

In [ ]:
def distortion_function(df):
  df2=df.drop(columns=['Pregnancies','DiabetesPedigreeFunction'])
  #df2 = df.copy()
  df2['BMI'] = np.where(df2['BMI'].between(18,25), 1, 0) # 1 valore giusto, 0 anomalo
  df2['Glucose'] = np.where(df2['Glucose'] < 120, 1, 0)
  df2["BloodPressure"]=np.where(df2["BloodPressure"].between(60,80),1,0)
  df2["SkinThickness"]=np.where(df2["SkinThickness"].between(22,24),1,0)
  df2["Insulin"]=np.where(df2["Insulin"].between(5,25),1,0)
  return df2

In [ ]:
def aif360_op(config):
  mitigation = 'aif360-op'
  df, target_variable, sensible_attribute, path_to_project, n_splits, models, n_estimators, random_seed = unpack_config(config)

  df_op = distortion_function(df)

  data_orig_aif_op = BinaryLabelDataset(
    favorable_label = 1,
    unfavorable_label = 0,
    df = df_op.copy(),
    label_names = [target_variable],
    protected_attribute_names = [sensible_attribute],
    metadata = default_mappings)

  optim_options = {
    "distortion_fun": get_distortion_diabetes,
    "epsilon": 0.05,
    "clist": [0.99, 1.99, 2.99],
    "dlist": [.1, 0.05, 0]
  }

  predictions_and_tests = {}
  OP = OptimPreproc(OptTools, optim_options)
  dataset_orig_train, dataset_orig_test = data_orig_aif_op.split([0.7], shuffle=True)
  OP_fitted= OP.fit(dataset_orig_train)
  transf_dataset = OP_fitted.transform(data_orig_aif_op, transform_Y=True)
  mit_aif360_op = transf_dataset.convert_to_dataframe()[0]

  save_mitigated_dataset(mit_aif360_op,path_to_project,dataset_name, sensible_attribute, mitigation)

  predictions_and_tests = compute_predictions_and_tests(mit_aif360_op, target_variable, n_splits, models, n_estimators, random_seed)

  save_predictions_and_tests(predictions_and_tests, dataset_name, sensible_attribute, mitigation, path_to_project)

  return predictions_and_tests

In [ ]:
print(aif360_op(config))

KeyError: "['Pregnancies', 'DiabetesPedigreeFunction'] not found in axis"

##In-processing

Ogni tecnica genera un nuovo classificatore: no score and no mitigated dataset

### Prejudice remover
In-processing: dataset is equal to the original one, the predictor is fairer by construction and given (FIXED WITH VERSION 0.6.1 OF AIF360)

PARAMETER eta

NO MITIGATED DATASET

NO SCORE FUNCTION

MODEL ENFORCEMENT



In [ ]:
def aif360_pr(config):
  mitigation = 'aif360-pr'
  df, target_variable,sensible_attribute, path_to_project, n_splits, models, n_estimators, random_seed = unpack_config(config)

  predictions_and_tests = {}

  df_splitting = train_test_splitting(df, n_splits)
  pred_and_y = {}
  for i in range(0,n_splits):

    PR = PrejudiceRemover(eta=1.0, sensitive_attr=sensible_attribute, class_attr=target_variable)

    df_split = df_splitting[i]
    df_train = df_split['train']
    df_test = df_split['test']

    X_train, Y_train, X_test, Y_test, S_train, S_test = df_X_Y_split_S(df_train, df_test, target_variable, sensible_attribute)

    data_orig_aif_train = BinaryLabelDataset(favorable_label = 1,
      unfavorable_label = 0,
      df = df_train.copy(),
      label_names = [target_variable],
      protected_attribute_names = [sensible_attribute])

    data_orig_aif_test = BinaryLabelDataset(favorable_label = 1,
      unfavorable_label = 0,
      df = df_test.copy(),
      label_names = [target_variable],
      protected_attribute_names = [sensible_attribute])

    print("TRAIN")
    print(data_orig_aif_train)
    print("TEST")
    print(data_orig_aif_test)

    PR.fit(data_orig_aif_train)
    predictions = PR.predict(data_orig_aif_test)
    mit_aif360_pr = predictions.convert_to_dataframe()[0]

    y_pred = mit_aif360_pr[target_variable]
    pred_and_y[i] = {'y_test': Y_test.to_numpy(), 'y_pred': y_pred.to_numpy().astype(int), 's_test':  S_test}

    predicted_and_real_tests = pred_and_y
  print(predicted_and_real_tests)


  save_predictions_and_tests(predicted_and_real_tests, dataset_name, sensible_attribute, mitigation, path_to_project)
  return predicted_and_real_tests



In [ ]:
aif360_pr(config)

TRAIN

instance weights features                                      \
                                                                               
                                      ID RAZRIV K_SH_POST   AGE ZSN_A NITR_S   
instance names                                                                 
137                         1.0    402.0    0.0       0.0  67.0   0.0    0.0   
138                         1.0    557.0    0.0       0.0  55.0   0.0    0.0   
139                         1.0    591.0    0.0       0.0  46.0   0.0    1.0   
140                         1.0   1110.0    0.0       0.0  79.0   0.0    0.0   
141                         1.0    240.0    0.0       0.0  60.0   0.0    0.0   
...                         ...      ...    ...       ...   ...   ...    ...   
1367                        1.0    831.0    0.0       0.0  71.0   0.0    0.0   
1368                        1.0    906.0    0.0       0.0  51.0   0.0    0.0   
1369                        1.0   1355.0    0.0       0.0  64.0   0.0    1.0   
1370                        1.0    619.0    0.0       0.0  42.0   0.0    0.0   
1371                        1.0    749.0    0.0       0.0  64.0   0.0    0.0   

                                            ...                               \
                                            ...                                
               NA_R_1_n R_AB_3_n STENOK_AN  ... REC_IM zab_leg_02 MP_TP_POST   
instance names                              ...                                
137                 0.0      0.0       0.0  ...    0.0        0.0        0.0   
138                 0.0      0.0       3.0  ...    0.0        0.0        0.0   
139                 0.0      1.0       0.0  ...    1.0        0.0        0.0   
140                 4.0      0.0       1.0  ...    0.0        1.0        0.0   
141                 0.0      0.0       6.0  ...    0.0        0.0        0.0   
...                 ...      ...       ...  ...    ...        ...        ...   
1367                0.0      1.0       4.0  ...    1.0        0.0        0.0   
1368                0.0      0.0       0.0  ...    0.0        0.0        0.0   
1369                0.0      0.0       1.0  ...    0.0        0.0        0.0   
1370                1.0      0.0       0.0  ...    0.0        0.0        0.0   
1371                0.0      0.0       0.0  ...    0.0        0.0        0.0   

                                                                              \
                                                         protected attribute   
               lat_im IBS_POST nr_04 NOT_NA_2_n R_AB_2_n                 SEX   
instance names                                                                 
137               1.0      2.0   0.0        0.0      0.0                 1.0   
138               1.0      1.0   0.0        0.0      0.0                 1.0   
139               1.0      2.0   0.0        0.0      0.0                 0.0   
140               1.0      1.0   0.0        0.0      0.0                 1.0   
141               0.0      2.0   0.0        0.0      0.0                 1.0   
...               ...      ...   ...        ...      ...                 ...   
1367              1.0      1.0   0.0        0.0      0.0                 0.0   
1368              1.0      0.0   0.0        0.0      0.0                 1.0   
1369              1.0      2.0   0.0        0.0      0.0                 0.0   
1370              1.0      2.0   0.0        0.0      0.0                 1.0   
1371              1.0      2.0   0.0        0.0      0.0                 1.0   

               labels  
                       
                       
instance names         
137               0.0  
138               0.0  
139               0.0  
140               0.0  
141               0.0  
...               ...  
1367              0.0  
1368              0.0  
1369              0.0  
1370              0.0  
1371              0.0  

[1235 rows x 25 columns]

TEST

instance weights features                                      \
                                                                               
                                      ID RAZRIV K_SH_POST   AGE ZSN_A NITR_S   
instance names                                                                 
0                           1.0   1118.0    0.0       0.0  57.0   0.0    0.0   
1                           1.0    705.0    0.0       0.0  44.0   0.0    0.0   
2                           1.0   1329.0    0.0       0.0  59.0   0.0    0.0   
3                           1.0     32.0    0.0       0.0  65.0   0.0    0.0   
4                           1.0    278.0    0.0       0.0  74.0   1.0    0.0   
...                         ...      ...    ...       ...   ...   ...    ...   
132                         1.0    633.0    0.0       0.0  57.0   0.0    0.0   
133                         1.0     48.0    0.0       0.0  72.0   0.0    0.0   
134                         1.0    361.0    0.0       0.0  57.0   0.0    0.0   
135                         1.0   1210.0    0.0       0.0  65.0   0.0    0.0   
136                         1.0    829.0    0.0       0.0  53.0   0.0    0.0   

                                            ...                               \
                                            ...                                
               NA_R_1_n R_AB_3_n STENOK_AN  ... REC_IM zab_leg_02 MP_TP_POST   
instance names                              ...                                
0                   0.0      0.0       6.0  ...    0.0        1.0        0.0   
1                   0.0      0.0       0.0  ...    0.0        0.0        0.0   
2                   0.0      0.0       6.0  ...    1.0        0.0        0.0   
3                   0.0      0.0       0.0  ...    0.0        0.0        0.0   
4                   0.0      0.0       0.0  ...    0.0        1.0        0.0   
...                 ...      ...       ...  ...    ...        ...        ...   
132                 0.0      0.0       6.0  ...    0.0        0.0        0.0   
133                 0.0      0.0       2.0  ...    0.0        0.0        0.0   
134                 0.0      0.0       0.0  ...    0.0        0.0        0.0   
135                 0.0      0.0       4.0  ...    0.0        0.0        1.0   
136                 1.0      0.0       1.0  ...    0.0        0.0        0.0   

                                                                              \
                                                         protected attribute   
               lat_im IBS_POST nr_04 NOT_NA_2_n R_AB_2_n                 SEX   
instance names                                                                 
0                 0.0      1.0   0.0        0.0      0.0                 1.0   
1                 1.0      0.0   0.0        0.0      0.0                 1.0   
2                 1.0      2.0   0.0        0.0      0.0                 1.0   
3                 1.0      0.0   0.0        0.0      0.0                 1.0   
4                 0.0      0.0   0.0        0.0      0.0                 0.0   
...               ...      ...   ...        ...      ...                 ...   
132               0.0      1.0   0.0        0.0      0.0                 1.0   
133               0.0      1.0   0.0        0.0      0.0                 0.0   
134               1.0      2.0   0.0        0.0      0.0                 1.0   
135               1.0      2.0   0.0        0.0      0.0                 1.0   
136               0.0      2.0   0.0        0.0      0.0                 1.0   

               labels  
                       
                       
instance names         
0                 0.0  
1                 0.0  
2                 0.0  
3                 0.0  
4                 0.0  
...               ...  
132               0.0  
133               0.0  
134               0.0  
135               0.0  
136               0.0  

[137 rows x 25 columns]

TRAIN

...

instance weights features                                      \
                                                                               
                                      ID RAZRIV K_SH_POST   AGE ZSN_A NITR_S   
instance names                                                                 
0                           1.0   1118.0    0.0       0.0  57.0   0.0    0.0   
1                           1.0    705.0    0.0       0.0  44.0   0.0    0.0   
2                           1.0   1329.0    0.0       0.0  59.0   0.0    0.0   
3                           1.0     32.0    0.0       0.0  65.0   0.0    0.0   
4                           1.0    278.0    0.0       0.0  74.0   1.0    0.0   
...                         ...      ...    ...       ...   ...   ...    ...   
1228                        1.0    670.0    0.0       0.0  63.0   0.0    0.0   
1229                        1.0   1142.0    0.0       0.0  55.0   0.0    0.0   
1230                        1.0    323.0    0.0       0.0  69.0   1.0    1.0   
1231                        1.0    101.0    0.0       0.0  50.0   0.0    0.0   
1232                        1.0   1395.0    0.0       0.0  74.0   0.0    1.0   

                                            ...                               \
                                            ...                                
               NA_R_1_n R_AB_3_n STENOK_AN  ... REC_IM zab_leg_02 MP_TP_POST   
instance names                              ...                                
0                   0.0      0.0       6.0  ...    0.0        1.0        0.0   
1                   0.0      0.0       0.0  ...    0.0        0.0        0.0   
2                   0.0      0.0       6.0  ...    1.0        0.0        0.0   
3                   0.0      0.0       0.0  ...    0.0        0.0        0.0   
4                   0.0      0.0       0.0  ...    0.0        1.0        0.0   
...                 ...      ...       ...  ...    ...        ...        ...   
1228                0.0      0.0       6.0  ...    0.0        0.0        0.0   
1229                0.0      0.0       2.0  ...    1.0        0.0        0.0   
1230                0.0      0.0       0.0  ...    0.0        0.0        0.0   
1231                0.0      0.0       2.0  ...    0.0        0.0        0.0   
1232                1.0      1.0       6.0  ...    0.0        0.0        0.0   

                                                                              \
                                                         protected attribute   
               lat_im IBS_POST nr_04 NOT_NA_2_n R_AB_2_n                 SEX   
instance names                                                                 
0                 0.0      1.0   0.0        0.0      0.0                 1.0   
1                 1.0      0.0   0.0        0.0      0.0                 1.0   
2                 1.0      2.0   0.0        0.0      0.0                 1.0   
3                 1.0      0.0   0.0        0.0      0.0                 1.0   
4                 0.0      0.0   0.0        0.0      0.0                 0.0   
...               ...      ...   ...        ...      ...                 ...   
1228              1.0      1.0   0.0        0.0      0.0                 1.0   
1229              0.0      2.0   0.0        0.0      0.0                 0.0   
1230              1.0      0.0   0.0        0.0      0.0                 1.0   
1231              1.0      1.0   0.0        0.0      0.0                 1.0   
1232              0.0      2.0   0.0        0.0      0.0                 0.0   

               labels  
                       
                       
instance names         
0                 0.0  
1                 0.0  
2                 0.0  
3                 0.0  
4                 0.0  
...               ...  
1228              0.0  
1229              0.0  
1230              0.0  
1231              0.0  
1232              0.0  

[1233 rows x 25 columns]

TEST

instance weights features                                      \
                                                                               
                                      ID RAZRIV K_SH_POST   AGE ZSN_A NITR_S   
instance names                                                                 
1233                        1.0    700.0    0.0       0.0  67.0   0.0    0.0   
1234                        1.0    313.0    0.0       0.0  27.0   0.0    0.0   
1235                        1.0    541.0    0.0       0.0  66.0   0.0    0.0   
1236                        1.0    492.0    0.0       0.0  32.0   0.0    0.0   
1237                        1.0   1344.0    0.0       0.0  70.0   0.0    0.0   
...                         ...      ...    ...       ...   ...   ...    ...   
1365                        1.0    304.0    0.0       0.0  64.0   0.0    1.0   
1366                        1.0   1130.0    0.0       0.0  47.0   0.0    0.0   
1367                        1.0    831.0    0.0       0.0  71.0   0.0    0.0   
1368                        1.0    906.0    0.0       0.0  51.0   0.0    0.0   
1369                        1.0   1355.0    0.0       0.0  64.0   0.0    1.0   

                                            ...                               \
                                            ...                                
               NA_R_1_n R_AB_3_n STENOK_AN  ... REC_IM zab_leg_02 MP_TP_POST   
instance names                              ...                                
1233                0.0      0.0       1.0  ...    0.0        0.0        0.0   
1234                0.0      0.0       1.0  ...    0.0        0.0        0.0   
1235                0.0      0.0       5.0  ...    0.0        0.0        0.0   
1236                0.0      0.0       1.0  ...    0.0        0.0        0.0   
1237                1.0      2.0       6.0  ...    1.0        0.0        0.0   
...                 ...      ...       ...  ...    ...        ...        ...   
1365                1.0      0.0       6.0  ...    0.0        0.0        0.0   
1366                0.0      0.0       2.0  ...    0.0        0.0        0.0   
1367                0.0      1.0       4.0  ...    1.0        0.0        0.0   
1368                0.0      0.0       0.0  ...    0.0        0.0        0.0   
1369                0.0      0.0       1.0  ...    0.0        0.0        0.0   

                                                                              \
                                                         protected attribute   
               lat_im IBS_POST nr_04 NOT_NA_2_n R_AB_2_n                 SEX   
instance names                                                                 
1233              1.0      1.0   0.0        1.0      0.0                 0.0   
1234              1.0      2.0   0.0        0.0      0.0                 1.0   
1235              0.0      1.0   0.0        0.0      0.0                 1.0   
1236              1.0      1.0   0.0        0.0      0.0                 0.0   
1237              0.0      2.0   0.0        1.0      1.0                 1.0   
...               ...      ...   ...        ...      ...                 ...   
1365              0.0      1.0   0.0        0.0      0.0                 1.0   
1366              1.0      2.0   0.0        0.0      0.0                 1.0   
1367              1.0      1.0   0.0        0.0      0.0                 0.0   
1368              1.0      0.0   0.0        0.0      0.0                 1.0   
1369              1.0      2.0   0.0        0.0      0.0                 0.0   

               labels  
                       
                       
instance names         
1233              0.0  
1234              0.0  
1235              0.0  
1236              0.0  
1237              0.0  
...               ...  
1365              0.0  
1366              0.0  
1367              0.0  
1368              0.0  
1369              0.0  

[137 rows x 25 columns]

{
    0: {
        'y_test': array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0]),
        'y_pred': array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0]),
        's_test': array([1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1,
       1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0,
       1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1,
       1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1,
       0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0,
       1, 0, 1, 1, 1])
    },
    1: {
        'y_test': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0]),
        'y_pred': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0]),
        's_test': array([1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1,
       1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1,
       1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       0, 1, 0, 1, 1])
    },
    2: {
        'y_test': array([0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0]),
        'y_pred': array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, [1;

{0: {'y_test': array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
         0, 0, 0, 0, 0]),
  'y_pred': array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0]),
  's_test': array([1, 1, 

### Exponentiated Gradient Reduction
In-processing: dataset is equal to the original one, the predictor is fairer by construction and given

PARAMETERS constraint e estimator

NO MITIGATED DATASET

NO SCORE FUNCTION

MODEL ENFORCEMENT

In [ ]:
def aif360_er(config):
  mitigation = 'aif360-er'
  df, target_variable,sensible_attribute, path_to_project, n_splits, models, n_estimators, random_seed = unpack_config(config)
  df_splitting = train_test_splitting(df, n_splits)
  pred_and_y = {}
  predictions_and_tests = {}
  for i in range(0,n_splits):

    df_split = df_splitting[i]
    df_train = df_split['train']
    df_test = df_split['test']

    X_train, Y_train, X_test, Y_test, S_train, S_test = df_X_Y_split_S(df_train, df_test, target_variable, sensible_attribute)

    data_orig_aif_train = BinaryLabelDataset(favorable_label = 1,
      unfavorable_label = 0,
      df = df_train.copy(),
      label_names = [target_variable],
      protected_attribute_names = [sensible_attribute])

    data_orig_aif_test = BinaryLabelDataset(favorable_label = 1,
      unfavorable_label = 0,
      df = df_test.copy(),
      label_names = [target_variable],
      protected_attribute_names = [sensible_attribute])

    #lmod = DecisionTreeClassifier(max_depth=None)
    lmod = LogisticRegression(solver='lbfgs', max_iter=1000)

    exp_grad_red = ExponentiatedGradientReduction(estimator=lmod, constraints='DemographicParity', drop_prot_attr=False)
    exp_grad_red.fit(data_orig_aif_train)
    predictions = exp_grad_red.predict(data_orig_aif_test)

    mit_aif360_er = predictions.convert_to_dataframe()[0]
    y_pred = mit_aif360_er[target_variable]
    pred_and_y[i] = {'y_test': Y_test.to_numpy(), 'y_pred': y_pred.to_numpy().astype(int), 's_test':  S_test}

  predictions_and_tests = pred_and_y

  print(predictions_and_tests)
  save_predictions_and_tests(predictions_and_tests, dataset_name, sensible_attribute, mitigation, path_to_project)
  return predictions_and_tests


In [ ]:
aif360_er(config)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/fairlearn/reductions/_moments/utility_parity.py:185: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updatin

{
    0: {
        'y_test': array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0]),
        'y_pred': array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0]),
        's_test': array([1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1,
       1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0,
       1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1,
       1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1,
       0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0,
       1, 0, 1, 1, 1])
    },
    1: {
        'y_test': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0]),
        'y_pred': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0]),
        's_test': array([1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1,
       1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1,
       1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       0, 1, 0, 1, 1])
    },
    2: {
        'y_test': array([0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0]),
        'y_pred': array([0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, [1;

{0: {'y_test': array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
         0, 0, 0, 0, 0]),
  'y_pred': array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
         0, 0, 0, 0, 0]),
  's_test': array([1, 1, 

### Adversarial Debiasing UGUALE A FAIRLEARN (ORIGINALE)
In-processing: dataset is equal to the original one, the predictor is fairer by construction and given

PARAMETER SESSION

NO MITIGATED DATASET

NO SCORE FUNCTION

MODEL ENFORCEMENT

In [ ]:
def aif360_ad(config):
  mitigation = 'aif360-ad'
  df, target_variable,sensible_attribute, path_to_project, n_splits, models, n_estimators, random_seed = unpack_config(config)

  predictions_and_tests = {}

  df_splitting = train_test_splitting(df, n_splits)
  pred_and_y = {}
  for i in range(0,n_splits):
    # Create a new session for each fold
    sess = tf.Session()
    # Added a unique scope name using the fold index
    AD = AdversarialDebiasing(privileged_groups = privileged_groups,
                        unprivileged_groups = unprivileged_groups,
                        scope_name=f'plain_classifier_{i}', # Changed scope name
                        debias=False,
                        sess=sess)

    df_split = df_splitting[i]
    df_train = df_split['train']
    df_test = df_split['test']

    X_train, Y_train, X_test, Y_test, S_train, S_test = df_X_Y_split_S(df_train, df_test, target_variable, sensible_attribute)

    data_orig_aif_train = BinaryLabelDataset(favorable_label = 1,
      unfavorable_label = 0,
      df = df_train.copy(),
      label_names = [target_variable],
      protected_attribute_names = [sensible_attribute])

    data_orig_aif_test = BinaryLabelDataset(favorable_label = 1,
      unfavorable_label = 0,
      df = df_test.copy(),
      label_names = [target_variable],
      protected_attribute_names = [sensible_attribute])

    AD.fit(data_orig_aif_train)

    predictions = AD.predict(data_orig_aif_test)

    mit_aif360_ad = predictions.convert_to_dataframe()[0]
    y_pred = mit_aif360_ad[target_variable]
    pred_and_y[i] = {'y_test': Y_test.to_numpy(), 'y_pred': y_pred.to_numpy().astype(int), 's_test':  S_test}

    predicted_and_real_tests = pred_and_y

    # Close the session after each fold
    sess.close()
  print(predicted_and_real_tests)
  save_predictions_and_tests(predicted_and_real_tests, dataset_name, sensible_attribute, mitigation, path_to_project)
  return predicted_and_real_tests

In [ ]:
aif360_ad(config)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


epoch 0; iter: 0; batch classifier loss: 9.940632
epoch 1; iter: 0; batch classifier loss: 10.535904
epoch 2; iter: 0; batch classifier loss: 9.635578
epoch 3; iter: 0; batch classifier loss: 5.145433
epoch 4; iter: 0; batch classifier loss: 6.067901
epoch 5; iter: 0; batch classifier loss: 5.976520
epoch 6; iter: 0; batch classifier loss: 6.498205
epoch 7; iter: 0; batch classifier loss: 4.022974
epoch 8; iter: 0; batch classifier loss: 4.141737
epoch 9; iter: 0; batch classifier loss: 2.808309
epoch 10; iter: 0; batch classifier loss: 2.567886
epoch 11; iter: 0; batch classifier loss: 5.272051
epoch 12; iter: 0; batch classifier loss: 3.766874
epoch 13; iter: 0; batch classifier loss: 7.169480
epoch 14; iter: 0; batch classifier loss: 3.553160
epoch 15; iter: 0; batch classifier loss: 2.052354
epoch 16; iter: 0; batch classifier loss: 3.260234
epoch 17; iter: 0; batch classifier loss: 4.515597
epoch 18; iter: 0; batch classifier loss: 1.590971
epoch 19; iter: 0; batch classifier loss

{
    0: {
        'y_test': array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0]),
        'y_pred': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0]),
        's_test': array([1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1,
       1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0,
       1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1,
       1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1,
       0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0,
       1, 0, 1, 1, 1])
    },
    1: {
        'y_test': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0]),
        'y_pred': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0]),
        's_test': array([1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1,
       1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1,
       1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       0, 1, 0, 1, 1])
    },
    2: {
        'y_test': array([0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0]),
        'y_pred': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, [1;

{0: {'y_test': array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
         0, 0, 0, 0, 0]),
  'y_pred': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0]),
  's_test': array([1, 1, 

## Post-processing

In the first phase there is the model building: the same for Calibrated Equalized Post-processing, Equalize Odds Post_processing and Reject Option Classification


In [ ]:
def compute_predictions(data_orig_aif, model):
  # Get the dataset and split into train and validation set
  dataset_orig_train, data_orig_valid = data_orig_aif.split([0.5], shuffle=True, seed=random_seed)

  dataset_orig_train_pred = dataset_orig_train.copy(deepcopy=True)
  data_orig_valid_pred = data_orig_valid.copy(deepcopy=True)

  X_train = dataset_orig_train.features
  y_train = dataset_orig_train.labels.ravel()

  model.fit(X_train, y_train)
  y_train_pred = model.predict(X_train)

  # positive class index
  pos_ind = np.where(model.classes_ == dataset_orig_train.favorable_label)[0][0]

  dataset_orig_train_pred.labels = y_train_pred

  X_valid = data_orig_valid_pred.features
  y_valid = data_orig_valid_pred.labels
  data_orig_valid_pred.scores = model.predict_proba(X_valid)[:,pos_ind].reshape(-1,1)

  return data_orig_valid, data_orig_valid_pred

### Calibrated Equalized Post-Processing

Post-processing: model's labels are changes, mitigated dataset is equal to the original one

PARAMETER cost_constraint

NO MITIGATED DATASET

NO EMBEDDED SCORE FUNCTION

NO MODEL ENFORCEMENT

In [ ]:
def aif360_ce(config):
  mitigation = 'aif360-ce'
  df, target_variable,sensible_attribute, path_to_project, n_splits, models, n_estimators, random_seed = unpack_config(config)

  predictions_and_tests = {}
  for model_name in models:
    clf = models[model_name]
    CEPP = CalibratedEqOddsPostprocessing(unprivileged_groups= unprivileged_groups, privileged_groups = privileged_groups, cost_constraint='weighted', seed=random_seed)

    df_splitting = train_test_splitting(df, n_splits)
    pred_and_y = {}
    for i in range(0,n_splits):
      df_split = df_splitting[i]
      df_train = df_split['train']
      df_test = df_split['test']

      X_train, Y_train, X_test, Y_test = df_X_Y_split(df_train, df_test, target_variable)
      S_test = df_test[sensible_attribute].values

      data_orig_aif_train = BinaryLabelDataset(favorable_label = 1,
      unfavorable_label = 0,
      df = df_train.copy(),
      label_names = [target_variable],
      protected_attribute_names = [sensible_attribute])

      data_orig_aif_test = BinaryLabelDataset(favorable_label = 1,
      unfavorable_label = 0,
      df = df_test.copy(),
      label_names = [target_variable],
      protected_attribute_names = [sensible_attribute])

      data_orig_valid, data_orig_valid_pred = compute_predictions(data_orig_aif_train, clf)

      cpp = CEPP.fit(data_orig_valid, data_orig_valid_pred)
      predictions = cpp.predict(data_orig_aif_test)
      mit_aif360_cpp = predictions.convert_to_dataframe()[0]
      y_pred = mit_aif360_cpp[target_variable]
      pred_and_y[i] = {'y_test': Y_test.to_numpy(), 'y_pred': y_pred.to_numpy().astype(int), 's_test':  S_test}

      predictions_and_tests[model_name] = pred_and_y

  print(predictions_and_tests)

  save_predictions_and_tests(predictions_and_tests, dataset_name, sensible_attribute, mitigation, path_to_project)

  return predictions_and_tests

In [ ]:
aif360_ce(config)

### Equalize Odds Post-Processing
Post-processing: model's labels are changes, mitigated dataset is equal to the original one (Fixed updating AIF360 Version)

NO PARAMETER

NO MITIGATED DATASET

NO EMBEDDED SCORE FUNCTION

NO MODEL ENFORCEMENT

In [ ]:
def aif360_eo(config):
  mitigation = 'aif360-eo'
  df, target_variable,sensible_attribute, path_to_project, n_splits, models, n_estimators, random_seed = unpack_config(config)

  predictions_and_tests = {}
  for model_name in models:
    clf = models[model_name]
    EOPP = EqOddsPostprocessing(unprivileged_groups= unprivileged_groups, privileged_groups = privileged_groups, seed=random_seed)

    df_splitting = train_test_splitting(df, n_splits)
    pred_and_y = {}
    for i in range(0,n_splits):
      df_split = df_splitting[i]
      df_train = df_split['train']
      df_test = df_split['test']

      X_train, Y_train, X_test, Y_test = df_X_Y_split(df_train, df_test, target_variable)
      S_test = df_test[sensible_attribute].values

      data_orig_aif_train = BinaryLabelDataset(favorable_label = 1,
      unfavorable_label = 0,
      df = df_train.copy(),
      label_names = [target_variable],
      protected_attribute_names = [sensible_attribute])

      data_orig_aif_test = BinaryLabelDataset(favorable_label = 1,
      unfavorable_label = 0,
      df = df_test.copy(),
      label_names = [target_variable],
      protected_attribute_names = [sensible_attribute])

      data_orig_valid, data_orig_valid_pred = compute_predictions(data_orig_aif_train, clf)

      eopp = EOPP.fit(data_orig_valid, data_orig_valid_pred)
      predictions = eopp.predict(data_orig_aif_test)
      mit_aif360_eopp = predictions.convert_to_dataframe()[0]
      y_pred = mit_aif360_eopp[target_variable]
      pred_and_y[i] = {'y_test': Y_test.to_numpy(), 'y_pred': y_pred.to_numpy().astype(int), 's_test':  S_test}

      predictions_and_tests[model_name] = pred_and_y

  print(predictions_and_tests)

  save_predictions_and_tests(predictions_and_tests, dataset_name, sensible_attribute, mitigation, path_to_project)

  return predictions_and_tests

In [ ]:
aif360_eo(config)

### Reject Option Classification

Post-processing: model's labels are changes, mitigated dataset is equal to the original one

NO MITIGATED DATASET

PARAMETERS: low_class_thresh,high_class_thresh, num_class_thresh, num_ROC_margin, metric_name, metric_ub, metric_lb

NO EMBEDDED SCORE FUCTION

NO MODEL ENFORCEMENT

In [ ]:
def aif360_roc(config):
  mitigation = 'aif360-roc'
  df, target_variable,sensible_attribute, path_to_project, n_splits, models, n_estimators, random_seed = unpack_config(config)

  predictions_and_tests = {}
  for model_name in models:
    clf = models[model_name]
    ROC = RejectOptionClassification(unprivileged_groups=unprivileged_groups,
                                 privileged_groups=privileged_groups,
                                 low_class_thresh=0.01, high_class_thresh=0.99,
                                  num_class_thresh=100, num_ROC_margin=50,
                                  metric_name="Statistical parity difference", metric_ub=0.05, metric_lb=-0.05)

    df_splitting = train_test_splitting(df, n_splits)
    pred_and_y = {}
    for i in range(0,n_splits):
      df_split = df_splitting[i]
      df_train = df_split['train']
      df_test = df_split['test']

      X_train, Y_train, X_test, Y_test = df_X_Y_split(df_train, df_test, target_variable)
      S_test = df_test[sensible_attribute].values.astype(int)

      data_orig_aif_train = BinaryLabelDataset(favorable_label = 1,
      unfavorable_label = 0,
      df = df_train.copy(),
      label_names = [target_variable],
      protected_attribute_names = [sensible_attribute])

      data_orig_aif_test = BinaryLabelDataset(favorable_label = 1,
      unfavorable_label = 0,
      df = df_test.copy(),
      label_names = [target_variable],
      protected_attribute_names = [sensible_attribute])

      data_orig_valid, data_orig_valid_pred = compute_predictions(data_orig_aif_train, clf)

      roc = ROC.fit(data_orig_valid, data_orig_valid_pred)
      predictions = roc.predict(data_orig_aif_test)
      mit_aif360_roc = predictions.convert_to_dataframe()[0]
      y_pred = mit_aif360_roc[target_variable]
      pred_and_y[i] = {'y_test': Y_test.to_numpy(), 'y_pred': y_pred.to_numpy().astype(int), 's_test':  S_test}

      predictions_and_tests[model_name] = pred_and_y

  print(predictions_and_tests)

  save_predictions_and_tests(predictions_and_tests, dataset_name, sensible_attribute, mitigation, path_to_project)

  return predictions_and_tests

In [ ]:
aif360_roc(config)